In [1]:
import pandas as pd

In [2]:
# данные разных лет с разным кол-вом дней в месяце
df_20_29 = 'https://www.nskes.ru/upload/iblock/70a/PUNTS_Fevral-2020_fakt-na-Sayt.xlsx'

# Входные данные
try:
  xls = pd.ExcelFile(df_20_29) # тут должно быть имя входного файла
except:
  raise Exception('Код ошибки -1')
input_voltage = 1 # (1, 2, 3, 4 ==  ВН, СН I, СН II, НH)
input_power = 10001 # в КВт

# output_name = 'tst.txt' # имя выходного файла

# выбор уровня напряжения
if input_voltage == 1:
  voltage = 'ВН'
elif input_voltage == 2:
  voltage = 'СН I'
elif input_voltage == 3:
  voltage = 'СН II'
elif input_voltage == 4:
  voltage = 'НН'
else:
   raise Exception('Код ошибки -2')

low = 150
medium = 670
high = 10000


# выбор листа в зависимости от мощности
power = input_power
# отриц
if power < 0:
   raise Exception('Код ошибки -2')
# менее 150
elif (power < low) and (str(low) in xls.sheet_names[0]):
  df = pd.read_excel(xls, sheet_name = 0)

# менее 670
elif (power < medium) and (str(medium) in xls.sheet_names[0]):
  df = pd.read_excel(xls, sheet_name = 0)

# от 150 до 670
elif (low <= power < medium) and (str(medium) in xls.sheet_names[1] and str(low) in xls.sheet_names[1]):
  df = pd.read_excel(xls, sheet_name = 1)

# от 670 до 10000
elif (medium <= power < high) and (str(medium) in xls.sheet_names[1] and '10' in xls.sheet_names[1]):
  df = pd.read_excel(xls, sheet_name= 1)
elif (medium <= power < high) and (str(medium) in xls.sheet_names[2] and '10' in xls.sheet_names[2]):
  df = pd.read_excel(xls, sheet_name= 2)

# свыше 10000
elif (power >= high) and ('10' in xls.sheet_names[2]) and ('менее' in xls.sheet_names[2]):
  df = pd.read_excel(xls, sheet_name=2)
elif (power >= high) and ('10' in xls.sheet_names[3]) and ('менее' in xls.sheet_names[3]):
  df = pd.read_excel(xls, sheet_name=3)

# ошибка
else:
  raise Exception('Код ошибки -2')

**Блок задания функций**

In [7]:
# критерий №1 (matrix)
def find_criterion_1(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка за электрическую энергию предельного уровня нерегулируемых цен' in str(sub_df):
        row1 = row
        col1 = col
        flag = True
      if flag == True:
        break
    if flag == True:
      flag = False
      break
  # поиск напряжения
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if str(sub_df) == voltage:
        #print(sub_df)
        flag = True
        row1 = row
        col1 = col
      if flag == True:
        break
    if flag == True:
      flag = False
      break
  # поиск левой границы матрицы
  row_1 = []
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if sub_df == 1:
        for dop_row in range(0, 31):
          date_ver = df.iloc[(row+dop_row):(row+dop_row+1), col:col+1].values.sum()
          num_ver = df.iloc[(row+dop_row):(row+dop_row+1), col+1:col+2].values.sum()
          if (date_ver in range(1, 32)) and (num_ver > 0 or num_ver < 0 or num_ver == 0):
            row_1 += [dop_row+row]
            matrix_col_1 = col + 1
            flag = True
            global matrix_length
            matrix_length = len(row_1)
      if flag == True:
        matrix_row_start_1 = row_1[0]
        matrix_row_end_1 = row_1[-1] + 1
        break
    if flag == True:
      flag = False
      break
  return matrix_row_start_1, matrix_row_end_1, matrix_col_1



#критерий №2 (num)
def find_criterion_2(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка за мощность, приобретаемую потребителем' in str(sub_df):
        for dop_col in range(1, len(df.columns)):
          num_ver = df.iloc[row:row+1, (col+dop_col) : (col+dop_col+1)].values.sum()
          if (num_ver > 0 or num_ver < 0 or num_ver == 0):
            out_row_2 = row
            out_col_2 = dop_col+col
            flag = True
          if flag == True:
            break
      if flag == True:
        break  
    if flag == True:
      flag = False
      break
  return out_row_2, out_col_2



# критерий №3 (matrix)
def find_criterion_3(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка для превышения фактического почасового объема покупки электрической энергии' in str(sub_df):
        row1 = row
        col1 = col
        flag = True
      if flag == True:
        break
    if flag == True:
      flag = False
      break
  # поиск левой границы матрицы
  row_3 = []
  flag = False
  for row in range(row1, len(df)):
    for col in range(len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if sub_df == 1:
        for dop_row in range(matrix_length):
          date_ver = df.iloc[(row+dop_row):(row+dop_row+1), col:col+1].values.sum()
          if (date_ver in range(1, matrix_length+1)):
            row_3 += [dop_row+row]
            matrix_col_3 = col + 1
            flag = True
        if flag == True:
          matrix_row_start_3 = row_3[0]
          matrix_row_end_3 = row_3[-1] + 1
          break
    if flag == True:
      flag = False
      break
  return matrix_row_start_3, matrix_row_end_3, matrix_col_3



# критерий №4 (matrix)
def find_criterion_4(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка для превышения планового почасового объема покупки электрической энергии' in str(sub_df):
        row1 = row
        col1 = col
        flag = True
      if flag == True:
        break
    if flag == True:
      flag = False
      break
  # поиск левой границы матрицы
  row_4 = []
  flag = False
  for row in range(row1, len(df)):
    for col in range(len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if sub_df == 1:
        for dop_row in range(matrix_length):
          date_ver = df.iloc[(row+dop_row):(row+dop_row+1), col:col+1].values.sum()
          if (date_ver in range(1, matrix_length+1)):
            row_4 += [dop_row+row]
            matrix_col_4 = col + 1
            flag = True
        if flag == True:
          matrix_row_start_4 = row_4[0]
          matrix_row_end_4 = row_4[-1] + 1
          break
    if flag == True:
      flag = False
      break
  return matrix_row_start_4, matrix_row_end_4, matrix_col_4



#критерий №5 (num)
def find_criterion_5(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка для суммы плановых почасовых объемов покупки электрической энергии' in str(sub_df):
        for dop_col in range(1, len(df.columns)):
          num_ver = df.iloc[row:row+1, (col+dop_col) : (col+dop_col+1)].values.sum()
          if (num_ver > 0 or num_ver < 0 or num_ver == 0):
            out_row_5 = row
            out_col_5 = dop_col+col
            flag = True
          if flag == True:
            break
      if flag == True:
        break  
    if flag == True:
      flag = False
      break
  return out_row_5, out_col_5



#критерий №6 (num)
def find_criterion_6(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка для суммы абсолютных значений разностей фактических и плановых почасовых' in str(sub_df):
        for dop_col in range(1, len(df.columns)):
          num_ver = df.iloc[row:row+1, (col+dop_col) : (col+dop_col+1)].values.sum()
          if (num_ver > 0 or num_ver < 0 or num_ver == 0):
            out_row_6 = row
            out_col_6 = dop_col+col
            flag = True
          if flag == True:
            break
      if flag == True:
        break  
    if flag == True:
      flag = False
      break
  return out_row_6, out_col_6



# критерий №7 (num)
def find_criterion_7(row1=0, col1=0):
  flag = False
  for row in range(row1, len(df)):
    for col in range(col1, len(df.columns)):
      sub_df = df.iloc[row:row+1, col:col+1].values.sum()
      if 'Ставка тарифа на услуги по передаче электрической энергии за содержание' in str(sub_df):
        for dop_col in range(1, len(df.columns)):
          num_ver = df.iloc[row:row+1, (col+dop_col) : (col+dop_col+1)].values.sum()
          volt_ver = df.iloc[row-1:row, (col+dop_col) : (col+dop_col+1)].values.sum()
          if (num_ver > 0 or num_ver < 0 or num_ver == 0) and volt_ver == voltage:
            out_row_7 = row
            out_col_7 = dop_col+col
            flag = True
            if flag == True:
              break
        if flag == True:
          break
    if flag == True:
      flag = False
      break  
  return out_row_7, out_col_7

Альтернативный критерий №7

Return'ы функций критериев:


*   1, 3, 4  - № строки начала матрицы, № строки конца матрицы, № столбца начала матицы
*   2, 5, 6, 7 - № строки и № столбца числа



*   3 цк - 1, 2 (всего 2)
*   4 цк - 1, 2, 7 (всего 3)
*   5 цк - 1, 2, 3, 4, 5, 6 (всего 6)
*   6 цк - 1, 2, 3, 4, 5, 6, 7 (всего 7)


In [8]:
### блок поиска и выборки данных последовательно по ЦК и критериям, внутри ЦК 

# 3 ЦК (критерии 1,2):

## 1 матрица
r_3_1_s, r_3_1_e, c_3_1 = find_criterion_1()
out_3_1 =  df.iloc[r_3_1_s:r_3_1_e, c_3_1:c_3_1+24]
## 2 число
r_3_2, c_3_2 = find_criterion_2(r_3_1_e)
out_3_2 =  df.iloc[r_3_2:r_3_2+1, c_3_2:c_3_2+1]

# 4 ЦК (критерии 1, 2, 7)

## 1 матрица
r_4_1_s, r_4_1_e, c_4_1 = find_criterion_1(r_3_2+1)
out_4_1 =  df.iloc[r_4_1_s:r_4_1_e, c_4_1:c_4_1+24]
## 2 число
r_4_2, c_4_2 = find_criterion_2(r_4_1_e)
out_4_2 =  df.iloc[r_4_2:r_4_2+1, c_4_2:c_4_2+1]
# 7 число
r_4_7, c_4_7 = find_criterion_7(r_4_1_e)
out_4_7 =  df.iloc[r_4_7:r_4_7+1, c_4_7:c_4_7+1]

# 5 ЦК (критерии 1, 2, 3, 4, 5, 6) 

## 1 матрица
r_5_1_s, r_5_1_e, c_5_1 = find_criterion_1(r_4_7+1)
out_5_1 =  df.iloc[r_5_1_s:r_5_1_e, c_5_1:c_5_1+24]
## 2 число
r_5_2, c_5_2 = find_criterion_2(r_5_1_e)
out_5_2 =  df.iloc[r_5_2:r_5_2+1, c_5_2:c_5_2+1]
## 3 матрица
r_5_3_s, r_5_3_e, c_5_3 = find_criterion_3(r_5_1_e)
out_5_3 =  df.iloc[r_5_3_s:r_5_3_e, c_5_3:c_5_3+24]
## 4 матрица
r_5_4_s, r_5_4_e, c_5_4 = find_criterion_4(r_5_3_e)
out_5_4 =  df.iloc[r_5_4_s:r_5_4_e, c_5_4:c_5_4+24]
## 5 число
r_5_5, c_5_5 = find_criterion_5(r_5_4_e)
out_5_5 = df.iloc[r_5_5:r_5_5+1, c_5_5:c_5_5+1]
## 6 число
r_5_6, c_5_6 = find_criterion_6(r_5_4_e)
out_5_6 = df.iloc[r_5_6:r_5_6+1, c_5_6:c_5_6+1]

# 6 ЦК (критерии 1, 2, 3, 4, 5, 6, 7) 

## 1 матрица
r_6_1_s, r_6_1_e, c_6_1 = find_criterion_1(r_5_2+1)
out_6_1 =  df.iloc[r_6_1_s:r_6_1_e, c_6_1:c_6_1+24]
## 2 число
r_6_2, c_6_2 = find_criterion_2(r_6_1_e)
out_6_2 =  df.iloc[r_6_2:r_6_2+1, c_6_2:c_6_2+1]
## 3 матрица
r_6_3_s, r_6_3_e, c_6_3 = find_criterion_3(r_6_1_e)
out_6_3 =  df.iloc[r_6_3_s:r_6_3_e, c_6_3:c_6_3+24]
## 4 матрица
r_6_4_s, r_6_4_e, c_6_4 = find_criterion_4(r_6_3_e)
out_6_4 =  df.iloc[r_6_4_s:r_6_4_e, c_6_4:c_6_4+24]
## 5 число
r_6_5, c_6_5 = find_criterion_5(r_6_4_e)
out_6_5 = df.iloc[r_6_5:r_6_5+1, c_6_5:c_6_5+1]
## 6 число
r_6_6, c_6_6 = find_criterion_6(r_6_4_e)
out_6_6 = df.iloc[r_6_6:r_6_6+1, c_6_6:c_6_6+1]
## 7 число
r_6_7, c_6_7 = find_criterion_7(r_6_4_e)
out_6_7 = df.iloc[r_6_7:r_6_7+1, c_6_7:c_6_7+1]



# собранные в список выходные данные
out_3 = [out_3_1, out_3_2]
out_4 = [out_4_1, out_4_2, out_4_7]
out_5 = [out_5_1, out_5_2, out_5_3, out_5_4, out_5_5, out_5_6]
out_6 = [out_6_1, out_6_2, out_6_3, out_6_4, out_6_5, out_6_6, out_6_7]

In [36]:
# запись данных в файл

empty_df = pd.DataFrame()
empty_df.to_csv('Output.txt', header=None, index=False)

for out_data in out_3:
  out_data.to_csv('Output.txt', header=None, index=False, mode='a')

for out_data in out_4:
  out_data.to_csv('Output.txt', header=None, index=False, mode='a')

for out_data in out_5:
  out_data.to_csv('Output.txt', header=None, index=False, mode='a')

for out_data in out_6:
  out_data.to_csv('Output.txt', header=None, index=False, mode='a')